# **Made in America: The New Energy Investments Delivering Economic Benefits to Your District**

This dashboard tracks real-world private investments in clean energy and manufacturing across the U.S., based on data from the [Clean Investment Monitor](https://www.cleaninvestmentmonitor.org/), a joint project of the Rhodium Group and MIT’s Center for Energy & Environmental Policy Research.

Since 2021, **landmark federal policies—including new clean energy and manufacturing tax credits—have fueled billions in private investment.** These policies are attracting capital, strengthening domestic supply chains, and lowering energy costs—all while ensuring the U.S. leads in critical technologies like clean power, batteries, and advanced manufacturing.  

📍 **Why this matters for your district:**  

✅ **Billions in new investments** are already flowing into American factories, energy projects, and supply chains—driven by stable policy that incentivizes domestic production.  

✅ **Rolling back these incentives could put American jobs and manufacturing at risk**—stranding existing investments and discouraging new projects.  

✅ **Cutting support for energy expansion will drive up electricity prices** just as demand surges, particularly from data centers and AI. Keeping energy costs low requires scaling clean power and modernizing the grid.  

This isn’t just about energy—it’s about keeping manufacturing at home, securing America’s competitive edge, and creating jobs where they’re needed most.

In [1]:
import pandas as pd
import folium
import geopandas as gpd
from flask import Flask, request, render_template_string
import threading
import webbrowser
import numpy as np
from IPython.display import HTML, display
import warnings
import os
import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import HTML
warnings.filterwarnings('ignore')

# Load your projects data - adjust the path as needed for your Jupyter environment
facility_data_path = 'Data/extended_cim_download_2024_Q4/manufacturing_energy_and_industry_facility_metadata.csv'
actual_investment_data_path = 'Data/extended_cim_download_2024_Q4/congressional_district_actual_investment_manufacturing_energy_and_industry.csv'

# Load actual investment data
actual_investment_df = pd.read_csv(actual_investment_data_path, skiprows=4)
projects_df = pd.read_csv(facility_data_path, skiprows=5)
projects_df['Decarb_Sector'] = projects_df['Decarb_Sector'].replace("Clean Tech Manufacturing", "Manufacturing")

# === NEW: Define Tax Credit Mapping Based on the README ===
credit_map = {
    "Manufacturing": {
        "Batteries": "45X, 48C",
        "Critical Minerals": "45X, 48C",
        "Electrolyzers": "48C",
        "Solar": "45X, 48C",
        "Wind": "45X, 48C",
        "Fueling Equipment": "48C",
        "Zero Emission Vehicles": "45X, 48C"  # For component manufacturing
    },
    "Energy and Industry": {
        "Carbon Management": "45Q",
        "Cement": "45Q, 48C",
        "Clean Fuels": "45Q, 45Z, 48C",
        "Hydrogen": "45V",
        "Iron & Steel": "45Q, 48C",
        "Pulp & Paper": "45Q, 48C",
        "SAF": "40B, 45Z",
        "Storage": "48, 48E",
        "Solar": "45, 48, 45Y, 48E",
        "Wind": "45, 48, 45Y, 48E",
        "Nuclear": "45U, 45Y, 48E",
        "Other": "45, 48, 45Y, 48E"
    },
    "Retail": {
        "Distributed Electricity and Storage": "25D, 45, 48, 45Y, 48E",
        "Heat Pumps": "25C, 45L",
        "Zero Emission Vehicles": "30D, 45W"
    }
}

# Function to assign tax credits based on Segment and Technology.
def get_tax_credits(row):
    seg = row["Segment"]
    tech = row["Technology"]
    if seg in credit_map:
        return credit_map[seg].get(tech, "N/A")
    return "N/A"

# Add the new column to projects_df
projects_df["Relevant Credits"] = projects_df.apply(get_tax_credits, axis=1)

# Function to assign tax credits based on Segment and Technology.
def get_tax_credits(row):
    seg = row["Segment"]
    tech = row["Technology"]
    if seg in credit_map:
        return credit_map[seg].get(tech, "N/A")
    return "N/A"

# Add the new column to projects_df
projects_df["Relevant Credits"] = projects_df.apply(get_tax_credits, axis=1)

# (Optional) Quick check:
projects_df.head()

# Load the electricity price impacts data (global)
electricity_price_csv_path = "Data/Electricity_Price_Impacts_2026_2029.csv"
electricity_price_df = pd.read_csv(electricity_price_csv_path)

def get_geojson_path(cd_code):
    # Normalize CD code format (convert "WA-01" to "WA-1")
    if cd_code[-2] == "0":
        cd_code = cd_code[:-2] + cd_code[-1]
    # Load the congressional district GeoJSON file based on the user input 
    geojson_path = f"https://raw.githubusercontent.com/nickoneill/districts/refs/heads/gh-pages/cds/2022/{cd_code}/shape.geojson"
    # print(f"Fetching GeoJSON from: {geojson_path}")  # Debugging line
    return geojson_path

def generate_summary_stats(filtered_df, filtered_actual, state_code):
    """Generate summary statistics for the filtered dataframe"""    
    # Calculate summary statistics
    stats = {
        'Total Projects': len(filtered_df),
        'Total CAPEX ($M)': filtered_df['Estimated_Total_Facility_CAPEX'].sum(),
        'Average CAPEX per Project ($M)': filtered_df['Estimated_Total_Facility_CAPEX'].mean(),
        'Projects by Status': filtered_df['Current_Facility_Status'].value_counts().to_dict(),
        'Projects by Type': filtered_df['Project_Type'].value_counts().to_dict(),
        'Projects by Technology': filtered_df['Subcategory'].value_counts().to_dict(),
        'Earliest Announcement': filtered_df['Announcement_Date'].min(),
        'Latest Announcement': filtered_df['Announcement_Date'].max(),
        'Projects Under Construction': sum(filtered_df['Current_Facility_Status'] == 'Under Construction'),
        'Operating Projects': sum(filtered_df['Current_Facility_Status'] == 'Operating')
    }

    # Compute actual investment from the filtered actual investment DataFrame
    total_actual_investment = filtered_actual['Estimated_Actual_Quarterly_Expenditure'].sum()
    stats['Total Actual Investment'] = f"${total_actual_investment:,.0f}M" if total_actual_investment else "N/A"
    
    # Format currency values
    stats['Total CAPEX ($M)'] = f"${stats['Total CAPEX ($M)']:,.0f}M" if not pd.isna(stats['Total CAPEX ($M)']) else "N/A"
    stats['Average CAPEX per Project ($M)'] = f"${stats['Average CAPEX per Project ($M)']:,.1f}M" if not pd.isna(stats['Average CAPEX per Project ($M)']) else "N/A"

    # --- NEW: Extract Electricity Price Increase (2026) for Both Sectors ---
    for sector in ["Residential", "Commercial & Industrial"]:
        elec_row = electricity_price_df[
            (electricity_price_df["Year"] == 2026) &
            (electricity_price_df["Metric"] == "% Change") &
            (electricity_price_df["Sector"] == sector) &
            (electricity_price_df["State"] == state_code)
        ]
        if not elec_row.empty:
            stats[f"Electricity Price Increase ({sector})"] = f"{elec_row['Number'].values[0]}%"
        else:
            stats[f"Electricity Price Increase ({sector})"] = "N/A"
    return stats

def generate_dashboard(cd_code="WA-04"):
    """
    Generate an interactive dashboard for a specific congressional district
    """
    # Load the congressional district GeoJSON file
    geojson_path = get_geojson_path(cd_code)
    try:
        districts_gdf = gpd.read_file(geojson_path).to_crs(epsg=4326)
    except Exception as e:
        return HTML(f"<div class='alert alert-danger'>Invalid Congressional District Code '{cd_code}'. Please enter a valid one. Error: {str(e)}</div>")
    
    # Filter the DataFrames based on the selected CD code
    filtered_df = projects_df[projects_df['CD119_2024_Name'] == cd_code]
    filtered_actual = actual_investment_df[actual_investment_df['CD119_2024_Name'] == cd_code]

    # === NEW: Aggregate tax credits for the district ===
    def aggregate_tax_credits(credit_series):
        credits_set = set()
        for credit_str in credit_series:
            # Split by comma and strip whitespace.
            for c in credit_str.split(","):
                credits_set.add(c.strip())
        return ", ".join(sorted(credits_set))

    district_credits = aggregate_tax_credits(filtered_df["Relevant Credits"])

    # Define dictionary mapping tax credit codes to explanations
    tax_credit_explanations = {
        "40B": "Sustainable Aviation Fuel Credit – boosts production of low-GHG aviation fuels.",
        "45": "Clean Electricity Production Tax Credit – incentivizes low-emission power generation.",
        "45V": "Clean Hydrogen Production Tax Credit – supports production of low-carbon hydrogen.",
        "45X": "Advanced Manufacturing Production Credit – reduces costs for domestic clean tech manufacturing.",
        "45Y": "Tech-Neutral Clean Electricity Production Tax Credit – flexible support for various clean power generators.",
        "45Z": "Clean Fuels Credit – encourages renewable and low-carbon fuel production.",
        "48": "Clean Electricity Investment Tax Credit – lowers upfront costs for clean energy projects.",
        "48C": "Advanced Energy Project Credit – extra incentive for innovative manufacturing projects.",
        "48E": "Tech-Neutral Clean Electricity Investment Tax Credit – a tech-neutral twist on the ITC."
    }
    
    # Build the list items only for the credits present in the district_credits variable.
    explanation_items = ""
    for credit in district_credits.split(","):
        code = credit.strip()
        if code in tax_credit_explanations:
            explanation_items += f"<li><strong>{code}:</strong> {tax_credit_explanations[code]}</li>"
    
    # Now build the collapsible info block
    collapsible_info = f"""
    <button type="button" class="collapsible" style="background:#007bff;color:white;font-size:13px;padding:5px 10px;border:none;border-radius:5px;cursor:pointer;">
        Tax Credits Explained
    </button>
    <div class="content" style="padding:0 18px;display:none;overflow:hidden;background-color:#f1f1f1;border-radius:5px;margin-top:10px;">
        <ul>
            {explanation_items}
        </ul>
    </div>
    
    <script>
    var coll = document.getElementsByClassName("collapsible");
    for (var i = 0; i < coll.length; i++) {{
        coll[i].addEventListener("click", function() {{
            this.classList.toggle("active");
            var content = this.nextElementSibling;
            if (content.style.display === "block") {{
                content.style.display = "none";
            }} else {{
                content.style.display = "block";
            }}
        }});
    }}
    </script>
    """

    ### Mouseover functionality
    # collapsible_info = f"""
    # <button type="button" class="collapsible" style="background:#007bff; color:white; padding:10px; border:none; border-radius:5px; cursor:pointer;">
    #     Tax Credits Explained
    # </button>
    # <div class="content" style="padding:0 18px; display:none; overflow:hidden; background-color:#f1f1f1; border-radius:5px; margin-top:10px; min-width:250px;">
    #     <ul>
    #         {explanation_items}
    #     </ul>
    # </div>
    
    # <script>
    # var btn = document.querySelector(".collapsible");
    # var content = document.querySelector(".content");
    
    # btn.addEventListener("mouseover", function() {{
    #     content.style.display = "block";
    # }});
    # btn.addEventListener("mouseout", function() {{
    #     setTimeout(function() {{
    #         if (!btn.matches(':hover') && !content.matches(':hover')) {{
    #             content.style.display = "none";
    #         }}
    #     }}, 100);
    # }});
    
    # content.addEventListener("mouseover", function() {{
    #     content.style.display = "block";
    # }});
    # content.addEventListener("mouseout", function() {{
    #     setTimeout(function() {{
    #         if (!btn.matches(':hover') && !content.matches(':hover')) {{
    #             content.style.display = "none";
    #         }}
    #     }}, 100);
    # }});
    # </script>
    # """
    
    # Extract state code (assumes format like "WA-04")
    state_code = cd_code.split('-')[0]
    
    # Generate summary statistics including the electricity price increase
    summary_stats = generate_summary_stats(filtered_df, filtered_actual, state_code)
    
    # Rename columns for display
    column_rename_map = {
        'Company': 'Company',
        "Decarb_Sector": "Sector",
        'Subcategory': 'Technology',
        'Project_Type': 'Project Type',
        'Current_Facility_Status': 'Facility Status',
        'Estimated_Total_Facility_CAPEX': 'Total Estimated CAPEX ($M)',
        'Announcement_Date': 'Announcement Date',
        'Production_Date': 'Start of Production',
        'Construction_Start': 'Construction Start',
        'Construction_End': 'Construction Completion',
        'Relevant Credits': 'Relevant Credits'
    }
    
    filtered_df_renamed = filtered_df.rename(columns=column_rename_map)
    
    # Format CAPEX values
    filtered_df_renamed['Total Estimated CAPEX ($M)'] = (
        filtered_df_renamed['Total Estimated CAPEX ($M)']
        .apply(lambda x: f"{int(round(x)):,}" if pd.notna(x) else "N/A")
    )
    
    # Convert DataFrame to an HTML table
    table_html = filtered_df_renamed[list(column_rename_map.values())].to_html(index=False, classes="table table-striped")
    
    # Create a Folium map
    if len(filtered_df) > 0:
        map_center = [filtered_df_renamed['Latitude'].mean(), filtered_df_renamed['Longitude'].mean()]
    else:
        map_center = [39.8283, -98.5795]  # Default center: US
    m = folium.Map(location=map_center, zoom_start=6, tiles="cartodbpositron", width='100%', height=500)
    
    folium.GeoJson(
        districts_gdf,
        name="Congressional Districts",
        style_function=lambda feature: {
            "fillColor": "blue",
            "color": "black",
            "weight": 1,
            "fillOpacity": 0.2
        }
    ).add_to(m)
    
    # Add markers (existing code here)...
    for _, row in filtered_df_renamed.iterrows():
        capex_value = row['Total Estimated CAPEX ($M)']
        capex_numeric = float(capex_value.replace(",", "")) if capex_value != "N/A" else 0
        max_capex = filtered_df_renamed['Total Estimated CAPEX ($M)'].replace("N/A", 0).astype(str).str.replace(",", "").astype(float).max()
        circle_radius = (capex_numeric / max_capex) * 20 if max_capex > 0 else 5
        tooltip_text = """
        <b>{0}</b><br>
        <table style='border-collapse: collapse; width: 250px;'>
        {1} 
        </table>
        """.format(
            row['Company'],
            "".join(f"<tr><td><b>{column_rename_map.get(col, col)}:</b></td><td>{row[column_rename_map.get(col, col)]}</td></tr>" for col in column_rename_map.keys())
        )
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=circle_radius,
            popup=folium.Popup(tooltip_text, max_width=300),
            tooltip=tooltip_text,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.4,
            weight=1
        ).add_to(m)
    
    # Save map to a temporary HTML file for display
    map_file = 'temp_map.html'
    m.save(map_file)
    with open(map_file, 'r') as f:
        map_html = f.read()
    if os.path.exists(map_file):
        os.remove(map_file)
    
    # Create Overview section (all stats except electricity price increase)
    overview_html = f"""
    <div class="overview-stats">
        <h3>Overview of New Projects for Congressional District <u>{cd_code}</u></h3>
        <div class="summary-grid">
            <div class="stat-box">
                <h4>Total Projects</h4>
                <p class="stat-value">{summary_stats['Total Projects']}</p>
            </div>
            <div class="stat-box">
                <h4>Total CAPEX</h4>
                <p class="stat-value">{summary_stats['Total CAPEX ($M)']}</p>
            </div>
            <div class="stat-box">
                <h4>Avg. CAPEX per Project</h4>
                <p class="stat-value">{summary_stats['Average CAPEX per Project ($M)']}</p>
            </div>
            <div class="stat-box">
                <h4>Projects Under Construction</h4>
                <p class="stat-value">{summary_stats['Projects Under Construction']}</p>
            </div>
            <div class="stat-box">
                <h4>Operating Projects</h4>
                <p class="stat-value">{summary_stats['Operating Projects']}</p>
            </div>
            <div class="stat-box">
                <h4>Estimated Total Already Invested</h4>
                <p class="stat-value">{summary_stats['Total Actual Investment']}</p>
            </div>
        </div>
    </div>
    """
    
    # Create Electricity Price Increase section
    electricity_html = f"""
    <div class="electricity-stats" style="margin-top:20px;">
        <h3>Estimated Electricity Price Increases in <u>Your State</u> Over Next Three Years if Technology Neutral Electricity Tax Credits (45Y/48E) are Repealed<sup>1</sup></h3>
        <div class="summary-grid">
            <div class="stat-box">
                <h4>Residential</h4>
                <p class="stat-value">{summary_stats['Electricity Price Increase (Residential)']}</p>
            </div>
            <div class="stat-box">
                <h4>Commercial & Industrial</h4>
                <p class="stat-value">{summary_stats['Electricity Price Increase (Commercial & Industrial)']}</p>
            </div>
        </div>
        <p style="font-size: 0.9em; margin-top: 10px;">
            <sup>1</sup> Source: <a href="https://cebuyers.org/blog/ceba-report-repealing-clean-energy-tax-credits-would-raise-electricity-prices-for-american-families-and-job-creators-across-the-united-states/" target="_blank">
            <i>Electricity Price Impacts of Technology-Neutral Tax Incentives With Incremental Electricity Demand from Data Centers (NERA Economic Consulting)</i></a>
        </p>
    </div>
    """
    
    # Combine the overview and electricity sections into one summary block
    combined_summary = overview_html + electricity_html

    # --- NEW: Create Tax Credits Banner ---
    # (Assumes 'district_credits' has been computed as shown previously)
    tax_credits_banner = f"""
    <div class="tax-credits-banner" style="background-color:#e9ecef; padding:10px; border-radius:5px; margin-bottom:20px;">
        <strong>Recent energy and manufacturing tax credits supporting projects in <u>{cd_code}</u>:</strong>
        {district_credits}
    </div>
    """

    # --- NEW: Create Tax Credits Banner and Explanation ---
    # (Assumes 'district_credits' has been computed as shown previously)
    tax_credits_banner = f"""
    <div class="tax-credits-banner" style="background-color:#e9ecef; padding:10px; border-radius:5px; margin-bottom:20px;">
        <strong>Recent energy and manufacturing tax credits supporting projects in {cd_code}:</strong>
        {district_credits}
    </div>
    """

    # Now, reassemble the full HTML with the summary section above the map
    full_html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Congressional District Project Dashboard</title>
        <style>
            #map-container {{
                height: 500px;
                width: 100%;
                position: relative;
                overflow: hidden;
                margin-bottom: 20px;
            }}
            .summary-stats, .overview-stats, .electricity-stats {{
                margin: 20px 0;
                padding: 20px;
                background-color: #f8f9fa;
                border-radius: 5px;
            }}
            .summary-grid {{
                display: grid;
                grid-template-columns: repeat(auto-fill, minmax(150px, 1fr));
                gap: 10px;
                margin-bottom: 20px;
            }}
            .stat-box {{
                background-color: white;
                padding: 10px;
                border-radius: 5px;
                box-shadow: 0 1px 3px rgba(0,0,0,0.1);
                text-align: center;
            }}
            .stat-value {{
                font-size: 1.5rem;
                font-weight: bold;
                margin-bottom: 0;
            }}
            .breakdown-grid {{
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(150px, 1fr));
                gap: 10px;
            }}
            .control-panel {{
                margin-bottom: 20px;
                padding: 15px;
                background-color: #f8f9fa;
                border-radius: 5px;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
            }}
            .table-container {{
                overflow-x: auto;
            }}
        </style>
    </head>
    <body>
        <div class="container-fluid py-3">
            <h1 class="mb-4">Impact of Recent Policies</h1>
            {tax_credits_banner}
            {collapsible_info}
            {combined_summary}
            <h3 class="mb-3">Project Map</h3>
            <div id="map-container">
                {map_html}
            </div>
            <h3 class="mb-3">Project Details</h3>
            <div class="table-container">
                {table_html}
            </div>
        </div>
    </body>
    </html>
    """
    
    return HTML(full_html)

def create_interactive_dashboard():
    # Build list of valid district codes from your dataframe.
    cd_list = sorted({str(cd) for cd in projects_df['CD119_2024_Name'] if pd.notnull(cd)})

    cd_input = widgets.Combobox(
        placeholder='Type a district code',
        options=cd_list,
        value='WA-04',
        description='District:',
        ensure_option=True,
        continuous_update=False
    )

    update_button = widgets.Button(
        description='Update Dashboard',
        button_style='primary',
        tooltip='Click to update the dashboard'
    )
    update_button.style.button_color = '#007bff'

    output = widgets.Output()

    def on_button_clicked(b):
        with output:
            clear_output()
            try:
                dash = generate_dashboard(cd_input.value)
                display(dash)
            except Exception as e:
                print("Error generating dashboard:", e)

    update_button.on_click(on_button_clicked)

    input_area = widgets.HBox([cd_input, update_button])
    display(widgets.VBox([input_area, output]))

    # Immediately render the initial dashboard.
    with output:
        display(generate_dashboard(cd_input.value))

    display(HTML("""
        <script>
        require(["jquery"], function($) {
            $(document).ready(function() {
                setTimeout(function(){
                    var inputField = $("input.widget-combobox"); 
                    
                    if (inputField.length) {
                        inputField.off("keydown"); // Remove any old event bindings
                        
                        inputField.on("keydown", function(e) {
                            if (e.key === "Enter" || e.keyCode === 13) {
                                e.preventDefault();
                                e.stopImmediatePropagation();
                                $("button:contains('Update Dashboard')").trigger("click");
                            }
                        });
                    }
                }, 1000); // Ensures the widget fully loads before event binding
            });
        });
    </script>
    """))

# Run the dashboard
create_interactive_dashboard()